User specificiation
* start with 
* aliquot concentration typically 20mM
* number and volume of aliquots may change 
  * consult with Kyle for long-term reconsitution and aliquotting

In [1]:
## script to generate Scispot registrations for Working Compound Aliquots
## lkp 2023/06/26

import pandas as pd
import requests
import json
import sys
import datetime
import math

sys.stdout.write("Imported required packages")

api_key_file = 'G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data/scispot_api_key.txt'
API_KEY = open(api_key_file, 'r').readlines()[0]
#print(API_KEY)

Imported required packages

In [2]:
## functions

# lookup an entry by UUID
def find_entry_from_uuid(manager, uuid_list):
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/find-row"
    payload = {
        "apiKey": API_KEY,
        "manager": manager,
        "uuid": uuid_list
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

# fetch a Scispot entry based on the Sample ID
def fetch_entry_from_id(manager, sample_id):
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/find-row-by-id"
    payload = {
        "apiKey": API_KEY,
        "labsheet": manager,
        "id": sample_id
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

# turn json Scispot return into a pandas df
def json_to_pd_df(json_in):
    df_out = pd.DataFrame(data=json_in['rows'])
    #print(df_out)
    #print(json_in['headers'])
    df_out.columns = list(json_in['headers'])
    return df_out

# calculate reconstitution volume for stock from dry powder
## formula from Margaux's website - https://www.selleckchem.com/molaritycalculator.jsp
## Mass (g) = Concentration (mol/L) * Volume (L) * Molecular Weight (g/mol)
## rearranged formula solving for volume - 764.9815
## Volume (L) = Mass (g) / [Concentration (mol/L) * Molecular Weight (g/mol)]
## Volume (uL) = Volume (L) * (1L / 1000000.00 uL)
def calculate_reconstitution_volume(mg, molecular_weight, mmol):
    # convert to base units
    mass = float(mg) / 1000  # mg to grams
    desired_conc = float(mmol) / 1000 # mM to M
    
    volume_l = (mass / (desired_conc * molecular_weight_g_per_mol))
    volume_ul = volume_l * 1000000.00
    
    return(volume_ul)

sys.stdout.write("Imported required functions")

Imported required functions

In [3]:
# read in a list of Working Compound IDs
test_list = ['TAL0051_v1',
            'TAL0052_v1',
            'TAL0053_v1',
            'TAL0054_v1',
            'TAL0055_v1',
            'TAL0056_v1',
            'TAL0057_v1',
            'TAL0058_v1',
            'TAL0059_v1',
            'TAL0060_v1']

#test_list = ['TEST001_v1',
#            'TEST002_v1']

In [4]:
# molarity test for TAL0051_v1
mass_mg = 5.1
molecular_weight_g_per_mol = 333.3414
desired_conc_mm = 20

calculate_reconstitution_volume(mass_mg, molecular_weight_g_per_mol, desired_conc_mm)

764.9814874480036

In [8]:
# make children (aliquots) with parent linkage from UUIDs
# generate a new row for the protein manager
def new_wcompound_aliquot_row(wcompound_id):
    
    print(wcompound_id)
    print(fetch_entry_from_id("Working Compound Manager", wcompound_id))
    
    # lookup the stock Working Compound information from the provided UUID
    wcompound_stock = json_to_pd_df(fetch_entry_from_id("Working Compound Manager", wcompound_id))
    #print(wcompound_stock)
    wcompound_stock_id = wcompound_stock['Compound ID'].values[0]
    #print(wcompound_stock_id)
    stock_volume = wcompound_stock['Volume'].values
    #print(stock_volume)
    
    # volume not currently provided from the stock Working Compound entry, so must calculate!
    if not stock_volume.any():
        sys.stderr.write("No stock volume provided. Calculating from provided mass, molecular weight, and assuming 20mM desired concentration.\n")

        # look up the molecular weight from the parent Compound Manager entry
        compound_id = wcompound_stock['Parent Compound'].values[0]
        molecular_weight = float(json_to_pd_df(fetch_entry_from_id("Compound Manager", compound_id))['Molecular Weight'].values)
        
        # calculate volume required for 20mM stock concentration using the provided dry mass
        mass = float(wcompound_stock['Stock Quantity (powder)'].values[0])
        vol = calculate_reconstitution_volume(mass, molecular_weight, 20)
        sys.stderr.write("Reconstitution volume: %f\n" % vol)
    
    template_wcompound_row = [
        '', # Compound ID, parent compound ID + "_20mM_100ul_vial7"
        wcompound_stock['Supplier'].values[0], # Supplier
        wcompound_stock['Supplier ID'].values[0], # Supplier ID
        "", # Prepared Date
        '', # Volume
        "", # Volume Unit
        "", # Batch/Lot Number
        "", # Received Date
        "20", # Concentration TODO MAKE USER-DEFINED VARIABLE 
        "", # Concentration Unit
        "", # Stock Solvent
        '', # Expired Date
        '', # Managed By
        '', # Created By
        '', # Stock Quantity (powder)
        '', # Stock Quantity Unit
        '', # Solubility (mg/ml)
        '', # Location
        wcompound_stock['UUID'].values[0], # Parent Compound
        wcompound_stock['UUID'].values[0], # Parent
        '', # Children
        '', # Library
        '', # Group
        '' # Notes
    ]
    
    template_compound_row = pd.DataFrame([template_wcompound_row],
                                         columns = list(wcompound_stock)[1:])
    #print(template_compound_row)

    # based on the template values above, make the rows for aliquot "copies"
    new_wcompound_rows = pd.DataFrame(columns = list(wcompound_stock)[1:])
    #print(new_wcompound_rows)

    # iterate over all the "evenly divisible" aliquots first...
    for i in range(math.floor(vol / 100)):
        new_name = wcompound_stock_id + "_20mM_100ul_vial" + str(i+1)
        #print(new_name)
                
        new_row = template_compound_row
        new_row['Compound ID'] = new_name
        new_row['Volume'] = 100
        new_wcompound_rows = new_wcompound_rows.append([new_row])

    # ... then the "leftover" aliquot
    if not vol % 100 == 0:
        vol_leftover = vol - (math.floor(vol / 100)*100)
        #print(new_name, vol_leftover)
        new_name = wcompound_stock_id + "_20mM_" + str(math.floor(vol_leftover)) + "ul_vial" + str(math.ceil(vol / 100))
        
        new_row = template_compound_row
        new_row['Compound ID'] = new_name
        new_row['Volume'] = vol_leftover
        new_wcompound_rows = new_wcompound_rows.append([new_row])
        
    return(new_wcompound_rows)

# create a draft aliquot dataframe for manual checking before committing to Scispot
aliquot_df = pd.DataFrame()
for compound in test_list:
    aliquot_df = aliquot_df.append(new_wcompound_aliquot_row(compound))

aliquot_df

TAL0051_v1
{'rows': [['2f3dcb45-608c-42ba-ad0d-1a27e0617c0c', 'TAL0051_v1', 'Enamine', 'Z8396062591', '', '', '', 'TB23-052', '06/23/2023', '', '', '', '', '', '', '5.1', '', '', '', 'TAL0051', '', '', '', 'TBXT_TB23-052', '']], 'headers': ['UUID', 'Compound ID', 'Supplier', 'Supplier ID', 'Prepared Date', 'Volume', 'Volume Unit', 'Batch/Lot Number', 'Received Date', 'Concentration ', 'Concentration Unit', 'Stock Solvent', 'Expired Date', 'Managed By', 'Created By', 'Stock Quantity (powder)', 'Stock Quantity Unit', 'Solubility (mg/ml)', 'Location', 'Parent Compound', 'Parent', 'Children', 'Library', 'Group', 'Notes'], 'success': True}


No stock volume provided. Calculating from provided mass, molecular weight, and assuming 20mM desired concentration.
Reconstitution volume: 764.981487


TAL0052_v1
{'rows': [['bc9955cb-fe0a-428e-9474-16c1abb5ef70', 'TAL0052_v1', 'Enamine', 'Z8396062598', '', '', '', 'TB23-052', '06/23/2023', '', '', '', '', '', '', '5.1', '', '', '', 'TAL0052', '', '', '', 'TBXT_TB23-052', '']], 'headers': ['UUID', 'Compound ID', 'Supplier', 'Supplier ID', 'Prepared Date', 'Volume', 'Volume Unit', 'Batch/Lot Number', 'Received Date', 'Concentration ', 'Concentration Unit', 'Stock Solvent', 'Expired Date', 'Managed By', 'Created By', 'Stock Quantity (powder)', 'Stock Quantity Unit', 'Solubility (mg/ml)', 'Location', 'Parent Compound', 'Parent', 'Children', 'Library', 'Group', 'Notes'], 'success': True}


No stock volume provided. Calculating from provided mass, molecular weight, and assuming 20mM desired concentration.
Reconstitution volume: 764.981487


TAL0053_v1
{'rows': [['08573963-61c4-419e-af1d-7a528966ee32', 'TAL0053_v1', 'Enamine', 'Z8396062582', '', '', '', 'TB23-052', '06/23/2023', '', '', '', '', '', '', '5.1', '', '', '', 'TAL0053', '', '', '', 'TBXT_TB23-052', '']], 'headers': ['UUID', 'Compound ID', 'Supplier', 'Supplier ID', 'Prepared Date', 'Volume', 'Volume Unit', 'Batch/Lot Number', 'Received Date', 'Concentration ', 'Concentration Unit', 'Stock Solvent', 'Expired Date', 'Managed By', 'Created By', 'Stock Quantity (powder)', 'Stock Quantity Unit', 'Solubility (mg/ml)', 'Location', 'Parent Compound', 'Parent', 'Children', 'Library', 'Group', 'Notes'], 'success': True}


...

TAL0059_v1
{'rows': [['ffbfd060-fe2e-4df5-8411-7b739b6cf640', 'TAL0059_v1', 'Enamine', 'Z8396077732', '', '', '', 'TB23-052', '06/23/2023', '', '', '', '', '', '', '5.2', '', '', '', 'TAL0059', '', '', '', 'TBXT_TB23-052', '']], 'headers': ['UUID', 'Compound ID', 'Supplier', 'Supplier ID', 'Prepared Date', 'Volume', 'Volume Unit', 'Batch/Lot Number', 'Received Date', 'Concentration ', 'Concentration Unit', 'Stock Solvent', 'Expired Date', 'Managed By', 'Created By', 'Stock Quantity (powder)', 'Stock Quantity Unit', 'Solubility (mg/ml)', 'Location', 'Parent Compound', 'Parent', 'Children', 'Library', 'Group', 'Notes'], 'success': True}


No stock volume provided. Calculating from provided mass, molecular weight, and assuming 20mM desired concentration.
Reconstitution volume: 779.981124


TAL0060_v1
{'rows': [['51168daa-688f-43f7-b938-ecffc921cc5a', 'TAL0060_v1', 'Enamine', 'Z8396062585', '', '', '', 'TB23-052', '06/23/2023', '', '', '', '', '', '', '5.2', '', '', '', 'TAL0060', '', '', '', 'TBXT_TB23-052', '']], 'headers': ['UUID', 'Compound ID', 'Supplier', 'Supplier ID', 'Prepared Date', 'Volume', 'Volume Unit', 'Batch/Lot Number', 'Received Date', 'Concentration ', 'Concentration Unit', 'Stock Solvent', 'Expired Date', 'Managed By', 'Created By', 'Stock Quantity (powder)', 'Stock Quantity Unit', 'Solubility (mg/ml)', 'Location', 'Parent Compound', 'Parent', 'Children', 'Library', 'Group', 'Notes'], 'success': True}


No stock volume provided. Calculating from provided mass, molecular weight, and assuming 20mM desired concentration.
Reconstitution volume: 779.981124


,Compound ID,Supplier,Supplier ID,Prepared Date,Volume,Volume Unit,Batch/Lot Number,Received Date,Concentration,Concentration Unit,...,Stock Quantity (powder),Stock Quantity Unit,Solubility (mg/ml),Location,Parent Compound,Parent,Children,Library,Group,Notes
0,TAL0051_v1_20mM_100ul_vial1,Enamine,Z8396062591,,100,,,,20,,...,,,,,2f3dcb45-608c-42ba-ad0d-1a27e0617c0c,2f3dcb45-608c-42ba-ad0d-1a27e0617c0c,,,,
0,TAL0051_v1_20mM_100ul_vial2,Enamine,Z8396062591,,100,,,,20,,...,,,,,2f3dcb45-608c-42ba-ad0d-1a27e0617c0c,2f3dcb45-608c-42ba-ad0d-1a27e0617c0c,,,,
0,TAL0051_v1_20mM_100ul_vial3,Enamine,Z8396062591,,100,,,,20,,...,,,,,2f3dcb45-608c-42ba-ad0d-1a27e0617c0c,2f3dcb45-608c-42ba-ad0d-1a27e0617c0c,,,,
0,TAL0051_v1_20mM_100ul_vial4,Enamine,Z8396062591,,100,,,,20,,...,,,,,2f3dcb45-608c-42ba-ad0d-1a27e0617c0c,2f3dcb45-608c-42ba-ad0d-1a27e0617c0c,,,,
0,TAL0051_v1_20mM_100ul_vial5,Enamine,Z8396062591,,100,,,,20,,...,,,,,2f3dcb45-608c-42ba-ad0d-1a27e0617c0c,2f3dcb45-608c-42ba-ad0d-1a27e0617c0c,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,TAL0060_v1_20mM_100ul_vial4,Enamine,Z8396062585,,100,,,,20,,...,,,,,51168daa-688f-43f7-b938-ecffc921cc5a,51168daa-688f-43f7-b938-ecffc921cc5a,,,,
0,TAL0060_v1_20mM_100ul_vial5,Enamine,Z8396062585,,100,,,,20,,...,,,,,51168daa-688f-43f7-b938-ecffc921cc5a,51168daa-688f-43f7-b938-ecffc921cc5a,,,,
0,TAL0060_v1_20mM_100ul_vial6,Enamine,Z8396062585,,100,,,,20,,...,,,,,51168daa-688f-43f7-b938-ecffc921cc5a,51168daa-688f-43f7-b938-ecffc921cc5a,,,,
0,TAL0060_v1_20mM_100ul_vial7,Enamine,Z8396062585,,100,,,,20,,...,,,,,51168daa-688f-43f7-b938-ecffc921cc5a,51168daa-688f-43f7-b938-ecffc921cc5a,,,,


In [9]:
aliquot_df.to_csv('./draft_aliquot_entries_for_margaux.csv', index=False)

## TODO 
start below 

In [ ]:
# add a new row to the Working Compound Manager, i.e. aliquots
def add_wcompound_manager_entry(new_row):
    url = "https://api.scispot.io/tryingtofixcors/labsheets/add-rows"
    payload = {
        "apiKey": API_KEY,
        "manager": "Working Compound Manager",
        "rows": [new_row]
    }
    headers = {"Content-Type": "application/json"}
    #print(new_row)
    
    response = requests.request("POST", url, json=payload, headers=headers)
    print(response.text)

sys.stdout.write("Imported functions for manipulating Working Compound Manager")


In [ ]:
##
## GENERATE WORKING COMPOUND ALIQUOT ENTRIES
##

# read in a Scispot plate TSV export of Sample Manager
# ChESS_dbgen1_plate1 was done manually
#sample_fi = "G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data/Sample-2023-04-10T19_24_09.tsv" # ChESS_dbgen1_202212_plate1rep2 
#sample_fi = "G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data/Sample-2023-04-10T19_24_09.tsv" # ChESS_dbgen1_202212_plate1rep3
input_df = pd.read_table(sample_fi, sep='\t', engine="python")
input_df.tail()

# generate the new protein samples and add them to SciSpot Protein Manager
# note: uuid header string parses weird > "ï»¿uuid"
for uuid in input_df["ï»¿uuid"]:

    # define Protein fractions to be generated for each Sample
    extraction_frx = ['cytosol',
                     'chromatin',
                     'insoluble']
    
    # generate the Protein Manager row for each fraction
    for frx in extraction_frx:
        new_row = new_protein_row(uuid, frx)
    
        # add the row to the Protein Manager
        #add_protein_manager_entry(new_row)


##
## GENERATE PEPTIDE SAMPLES
##

# read in a Scispot plate TSV export of Protein Manager
protein_fi = "G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data/Protein-2023-04-11T20_08_57.tsv" # ChESS_dbgen_202212_plate1rep1_chromatin
input_df = pd.read_table(protein_fi, sep='\t', engine="python")
input_df.tail()

# generate the new peptide samples and add them to SciSpot Peptide Manager
# note: uuid header string parses weird > "ï»¿uuid"
for uuid in input_df["ï»¿uuid"]:
    
    # generate the Peptide Manager row
    new_row = new_peptide_row(uuid)
    #print(new_row)

    # add the row to the Peptide Manager
    #add_peptide_manager_entry(new_row)


In [ ]:
## make manifest
def make_new_plate():
    url = "https://api.scispot.io/tryingtofixcors/manifest/create"
    payload = {
        "apiKey": "f37df3be-56b8-4496-a7e3-3cf2fb5be033",
        "name": "Test Manifest Hello World",
                "template": "96-well plate",
         "plates": [
            {
                "template": "96-well plate",
                "wells": 96,
                "labsheets": [
                    {
                        "labsheet": "Protein Manager",
                        "items": [
                            {
                                "name": "JHC7_dbgen1_DMSO_2_chromatin",
                                "well": "A1"
                            },
                            {
                                "name": "CAOV3_dbgen1_JQ1_2_chromatin",
                                "well": "A2"
                            },
                            {
                                "name": "IMR32_dbgen1_JQ1_2_chromatin",
                                "well": "A3"
                            }
                        ]
                    }
                ]
            }
        ]
    }
    headers = {"Content-Type": "application/json"}
    response = requests.request("POST", url, json=payload, headers=headers)
    print(response.text)
    
make_new_plate()